In [2]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpdwjbxbgt".


In [7]:
%%cuda
#include <stdio.h>
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <chrono>

#define BLOCK_SIZE 4

// макрос, который используется с функциями CUDA во избежание повторения кода проверки успешности результата функции
#define SAFE_CALL(CallInstruction, error_str) { \
    cudaError_t cuerr = CallInstruction; \
    if (cuerr != cudaSuccess) { \
        fprintf(stderr, error_str, cudaGetErrorString(cuerr)); \
        return 0; \
    } \
}

// Печать матрицы
void printMatrix(float *matrix, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            printf("%f ", matrix[i * n + j]);
        }
        printf("\n");
    }
}

__global__ void kernelMatMul(float* a, float* b, float* c, int n) {
  // идентификатор нити по строке и столбцу
  int idx = blockDim.x*blockIdx.x+threadIdx.x; // строка
  int idy = blockDim.y*blockIdx.y+threadIdx.y; // столбец
  float sum = 0;
  // Проверяем, что текущие индексы строки и столбца находятся в пределах матрицы
  if (idx<n&&idy<n){
    // Выполняем умножение элементов строки матрицы a и столбца матрицы b
    for(int i = 0; i<n; i++) c[idx+idy*n]+= a[idy*n+i]*b[i*n+idx];
  }
}

void * matrix_mul_gpu(float* a, float* b, float* c, int N){
    int SIZE = N*N*sizeof(float);
    //Выделение памяти на девайсе
    float *a_dev = NULL;
    SAFE_CALL(cudaMalloc((void**)&a_dev, SIZE), "Cannot allocate device array for a: %s\n");
    float *b_dev = NULL;
    SAFE_CALL(cudaMalloc((void**)&b_dev, SIZE), "Cannot allocate device array for b: %s\n");
    float *c_dev = NULL;
    SAFE_CALL(cudaMalloc((void**)&c_dev, SIZE), "Cannot allocate device array for c: %s\n");

    // копирование матриц с хоста (cpu) на девайс (gpu)
    SAFE_CALL( cudaMemcpy(a_dev, a, SIZE, cudaMemcpyHostToDevice), "Cannot copy a array from host to device: %s\n");
    SAFE_CALL( cudaMemcpy(b_dev, b, SIZE, cudaMemcpyHostToDevice), "Cannot copy b array from host to device: %s\n");

    // задаем размер блоков + считаем количество блоков в сетке, так чтобы на каждый поток выделялся свой элемент массива
    dim3 threadsPerBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 blocksPerGrid((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (N + BLOCK_SIZE - 1) / BLOCK_SIZE);

    // Запуск ядра CUDA
    kernelMatMul<<<blocksPerGrid, threadsPerBlock>>>(a_dev, b_dev, c_dev, N);
    // Копирование результата обратно на хост
    cudaMemcpy(c, c_dev, SIZE, cudaMemcpyDeviceToHost);


    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);



}

void matrix_mul_cpu(float* a, float* b, float* c, int n){
    for(int i = 0; i<n; i++){
        for(int j = 0; j<n; j++){
            c[i*n+j]=0;
            for(int k = 0; k<n; k++){
                c[i*n+j] += a[k*n+j]*b[i*n+k];
            }
        }
    }
}

int main() {
    for(int j = 100; j<2001; j+=100){
      int N = j;

      float *a = (float *)calloc(N*N, sizeof(float));
      float *b = (float *)calloc(N*N, sizeof(float));
      float *c = (float *)calloc(N*N, sizeof(float));

      float *a_cpu = (float *)calloc(N*N, sizeof(float));
      float *b_cpu = (float *)calloc(N*N, sizeof(float));
      float *c_cpu = (float *)calloc(N*N, sizeof(float));

      for (int i = 0; i < N*N; ++i) a_cpu[i] = b_cpu[i] = a[i] = b[i] = i%N;


      // Вычисление на GPU
      auto start_gpu = std::chrono::high_resolution_clock::now();
      matrix_mul_gpu(a,b,c,N);
      auto end_gpu = std::chrono::high_resolution_clock::now();
      std::chrono::duration<double> elapsed_gpu = end_gpu - start_gpu;

      auto start_cpu = std::chrono::high_resolution_clock::now();
      matrix_mul_cpu(a_cpu,b_cpu,c_cpu,N);
      auto end_cpu = std::chrono::high_resolution_clock::now();
      std::chrono::duration<double> elapsed_cpu = end_cpu - start_cpu;

      std::cout << "Размер матрицы: " << j << "*" << j <<std::endl;
      std::cout << "Время вычисления на GPU: " << elapsed_gpu.count() << " секунд" << std::endl;
      std::cout << "Время вычисления на CPU: " << elapsed_cpu.count() << " секунд" << std::endl;

      float razn = 0;
      for(int i = 0; i<N*N; i++) razn = abs(c[i]-c_cpu[i]);

      std::cout << "Разница между абсолютными значениями матриц: " << razn << std::endl;
      std::cout << std::endl;


      // Освобождение памяти
      free(a);
      free(b);
      free(c);
      free(a_cpu);
      free(b_cpu);
      free(c_cpu);
    }

    return 0;
}

Размер матрицы: 100*100
Время вычисления на GPU: 0.0960096 секунд
Время вычисления на CPU: 0.00433438 секунд
Разница между абсолютными значениями матриц: 0

Размер матрицы: 200*200
Время вычисления на GPU: 0.000770773 секунд
Время вычисления на CPU: 0.0336811 секунд
Разница между абсолютными значениями матриц: 0

Размер матрицы: 300*300
Время вычисления на GPU: 0.00183119 секунд
Время вычисления на CPU: 0.118789 секунд
Разница между абсолютными значениями матриц: 0

Размер матрицы: 400*400
Время вычисления на GPU: 0.00359269 секунд
Время вычисления на CPU: 0.299333 секунд
Разница между абсолютными значениями матриц: 0

Размер матрицы: 500*500
Время вычисления на GPU: 0.00634749 секунд
Время вычисления на CPU: 0.679558 секунд
Разница между абсолютными значениями матриц: 0

Размер матрицы: 600*600
Время вычисления на GPU: 0.0107256 секунд
Время вычисления на CPU: 1.29156 секунд
Разница между абсолютными значениями матриц: 0

Размер матрицы: 700*700
Время вычисления на GPU: 0.0173902 секу